# Index

- Test Data
    - docs: `testing-meilisearch_docs_indexing_100.json`
    - keywords: `testing-meilisearch_keywords_indexing_100.json`

- Environment Variable
    - Meilisearch
        - `MEILISEARCH_URL`
        - `MEILISEARCH_DOCUMENTS_INDEX`
        - `MEILISEARCH_KEYWORDS_INDEX`
    - API Server
        - `FLASK_URL`
   
- API Endpoints
    - docs: `/v1/index`
    - keywords: `/v1/index/auto-complete`

In [1]:
import os
import json

import uuid

import requests
from urllib.parse import urljoin

In [2]:
# Variables

# Meilisearch
ms_url            = os.getenv('MEILISEARCH_URL')
ms_docs_index     = os.getenv('MEILISEARCH_DOCUMENTS_INDEX')
ms_keywords_index = os.getenv('MEILISEARCH_KEYWORDS_INDEX')

# API Server
flask_url = os.getenv('FLASK_URL')

# API Endpoints
index_docs_api_url     = urljoin(flask_url, '/v1/index')
index_keywords_api_url = urljoin(flask_url, '/v1/index/auto-complete')

test_docs_path     = "testing-meilisearch_docs_indexing_100.json"
test_keywords_path = "testing-meilisearch_keywords_indexing_100.json"

In [3]:
# Prepare POST Json

# docs
docs_index_data = {
    "index": ms_docs_index,
    "data": []
}
with open(test_docs_path, 'r') as infile: 
    raw_j = json.load(infile)
    docs_index_data['data'] = raw_j

# keywords
keywords_index_data = {
    'index': ms_keywords_index,
    'data': []
}    
with open(test_keywords_path, 'r') as infile: 
    raw_j = json.load(infile)
    keywords_index_data['data'] = raw_j

In [4]:
# Send POST Indexing Request to API server
requests.post(index_docs_api_url, json=docs_index_data)
requests.post(index_keywords_api_url, json=keywords_index_data)

<Response [200]>

# Search

- Environment Variable
    - Meilisearch
        - `MEILISEARCH_URL`
        - `MEILISEARCH_DOCUMENTS_INDEX`
        - `MEILISEARCH_KEYWORDS_INDEX`
    - API Server
        - `FLASK_URL`
   
- API Endpoints
    - docs: `/v1/search`
    - keywords: `/v1/search/auto-complete`

In [5]:
_query = "stm32"
_max   = 10

# search keywords
search_keywords_api_url = urljoin(
    flask_url, 
    '/v1/search/auto-complete?q={}&max={}'.format(
        _query,
        _max
    )
)

resp = requests.get(search_keywords_api_url)
print(resp.status_code)
print(json.dumps(resp.json(), indent=4, ensure_ascii=False))

200
{
    "query": "stm32",
    "result": [
        {
            "name": "stm32 書籍",
            "type": "keywords"
        },
        {
            "name": "stm32 教學",
            "type": "keywords"
        },
        {
            "name": "stm32 開發",
            "type": "keywords"
        },
        {
            "name": "stm32 開發板",
            "type": "keywords"
        },
        {
            "name": "stm32 生態",
            "type": "keywords"
        },
        {
            "name": "stm32cubemx 經驗",
            "type": "keywords"
        },
        {
            "name": "stm32cubemx 開發",
            "type": "keywords"
        },
        {
            "name": "stm32cubemx 配置",
            "type": "keywords"
        },
        {
            "name": "stm32cubeprogrammer",
            "type": "keywords"
        }
    ]
}


In [6]:
import meilisearch as ms
ms_client = ms.Client(ms_url)

query = "javascript"
page  = 0
limit = 2

ms_request = {
    'offset': page * limit,
    'limit':  limit,    
    'attributesToRetrieve': [
        'title',
        'href',
        'published_at_unix',
        'author_name',
        'author_href',
        'series_name',
        'series_href',
        'hashtags',
        'keywords',
        'reading_time'
    ],
    'attributesToHighlight': ['raw_hl_content'],
    'highlightPreTag': '<mark>',
    'highlightPostTag': '</mark>',
    'attributesToCrop': ['raw_hl_content:100'],
    'cropMarker': ''
}

resp = ms_client.index(ms_docs_index).search(
    query='函式', 
    opt_params=ms_request
)

print(json.dumps(resp, indent=4, ensure_ascii=False))

{
    "hits": [
        {
            "title": "Day 37 - 在 AWS Lambda 建立 OpenCV Layer",
            "hashtags": [
                "13th鐵人賽",
                "opencv",
                "aws lambda"
            ],
            "keywords": [
                "opencv",
                "opencv45func",
                "opencv45",
                "虛擬環境",
                "layer"
            ],
            "published_at_unix": 1636511427,
            "author_name": "yehchitsai",
            "series_name": "觀賞魚辨識的YOLO全餐",
            "href": "https://ithelp.ithome.com.tw/articles/10282533",
            "author_href": "https://ithelp.ithome.com.tw/users/20129510/ironman",
            "series_href": "https://ithelp.ithome.com.tw/users/20129510/ironman/4385",
            "reading_time": 11,
            "_formatted": {
                "title": "Day 37 - 在 AWS Lambda 建立 OpenCV Layer",
                "raw_hl_content": "這個練習，在 AWS Lambda 上完成。操作的流程如下：\n\n打包 Python 的 OpenCV <mark>函式</mark>庫\n在 AWS Lambda 建

In [7]:
query = "javascript"
page  = 0
limit = 2

# search docs
search_docs_api_url = urljoin(
    flask_url, 
    '/v1/search?q={q}&page={page}&limit={limit}'.format(
        q=query,
        page=page,
        limit=limit
    )
)
resp = requests.get(search_docs_api_url)
print(resp.status_code)
print(json.dumps(resp.json(), indent=4, ensure_ascii=False))

200
{
    "query": "javascript",
    "total": 13,
    "result": [
        {
            "position": 0,
            "title": "JavaScript 從 100% 繼續，再多程式語言也不是問題！",
            "link": "https://ithelp.ithome.com.tw/articles/10282368",
            "snippet": "大家好！\n自從系列開始到昨天，也已經流逝 45 天的時間了。\n這期間，總是會懷疑自己寫的文章夠不夠好，因為我所學到的知識好像仍不足拿來教學。\n於是，我開始強迫自己去學習覺得麻煩的事物，這也是我第一次下定決心認真對待某件事情。\n接下來，我們就來回顧過去 45 天的文章都介紹了什麼吧！\n\n系列的前 15 天，我們介紹了物件原型和相關方法，我也藉此重新找回被我收納在記憶藏寶箱中的",
            "lastmod": 1635695968,
            "about_this_result": {
                "author": {
                    "name": "Felix",
                    "link": "https://ithelp.ithome.com.tw/users/20141537/ironman"
                },
                "series": {
                    "name": "JavaScript 從 50% 開始，打造函式庫不是問題！",
                    "link": "https://ithelp.ithome.com.tw/users/20141537/ironman/4660"
                },
                "hashtags": [
                    "13th鐵人賽"
                ],
                "keywords": [
       